For all explanation questions, please refer pdf document which I have uploaded. Sorry for the inconvinience.

## Setup

In [284]:
import re
import string
import nltk
import math
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict
from nltk.util import bigrams
from nltk.stem import WordNetLemmatizer

In [285]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [286]:
# Download NLTK stop words (if not already downloaded)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [287]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Q 1: Import the movie reviews

In [288]:
# Open the file for reading
with open("drive/MyDrive/Colab Notebooks/NLP/Movie_Reviews.txt", "r") as file:
    # Initialize variables to store positive and negative reviews
    positive_reviews = []
    negative_reviews = []

    # Initialize a flag to keep track of whether we're reading positive or negative reviews
    reading_positive = False
    reading_negative = False

    # Iterate through the lines in the file
    for line in file:
        # Remove leading and trailing whitespace
        line = line.strip()

        # Check if the line indicates positive reviews
        if line == "Positive Reviews":
            reading_positive = True
            reading_negative = False
            next(file)  # Skip the additional line
        # Check if the line indicates negative reviews
        elif line == "Negative Reviews":
            reading_positive = False
            reading_negative = True
            next(file)  # Skip the additional line
        # If we're reading positive reviews, add the review to the positive_reviews list
        elif reading_positive and line:
            positive_reviews.append(line)
        # If we're reading negative reviews, add the review to the negative_reviews list
        elif reading_negative and line:
            if not line.startswith("==="):
                negative_reviews.append(line)
            else:
                break

In [289]:
# Print the extracted reviews
print("Positive Reviews:")
for review in positive_reviews:
    print(review)

Positive Reviews:
1. "Forrest Gump is an absolute masterpiece! Tom Hanks delivers an unforgettable performance, and the storytelling is heartwarming. This movie is a journey through life that will make you laugh, cry, and appreciate the simple beauties of existence."
2. "The Shawshank Redemption is a timeless classic. The powerful themes of hope, friendship, and redemption make it a must-watch. Morgan Freeman and Tim Robbins give exceptional performances in this brilliantly crafted film."
3. "The epic conclusion to The Lord of the Rings trilogy, The Return of the King, is a cinematic triumph. The breathtaking visuals, epic battles, and emotionally resonant story make it a monumental achievement in filmmaking."
4. "La La Land is a love letter to the magic of Hollywood and dreams. The chemistry between Ryan Gosling and Emma Stone is enchanting, and the music and dance sequences are a pure delight. A modern musical masterpiece."
5. "Wes Anderson's whimsical style shines in The Grand Budap

In [290]:
# Print the lengths of positive and negative reviews
print(f"Number of Positive Reviews: {len(positive_reviews)}")

Number of Positive Reviews: 12


In [291]:
print("\nNegative Reviews:")
for review in negative_reviews:
    print(review)


Negative Reviews:
1. "The Last Airbender is a disaster of a film adaptation. It butchers the beloved animated series with wooden acting, convoluted storytelling, and cringe-worthy special effects. A letdown for fans and newcomers alike."
2. "Another Transformers movie, and it's just more of the same: mindless explosions, incoherent plotlines, and an overreliance on CGI. This franchise desperately needs an overhaul."
3. "The Emoji Movie is a blatant cash grab with a shallow, uninspired plot. It fails to deliver clever humor or meaningful messages, making it a forgettable and disappointing animated film."
4. "Fifty Shades of Grey is a cringe-inducing attempt at romance. Poorly written dialogue and unconvincing chemistry between the leads make it an awkward and unfulfilling cinematic experience."
5. "Jack and Jill is an unbearable comedy that relies on stale humor and a painfully unfunny portrayal of Adam Sandler in a dual role. It's a prime example of lazy filmmaking."
6. "Superman IV i

In [292]:
# Print the lengths of positive and negative reviews
print(f"Number of Negative Reviews: {len(negative_reviews)}")

Number of Negative Reviews: 12


## Q 2: Pre-processing

In [293]:
# Initialize the stemmer and stopwords
stemmer = PorterStemmer()
# Instantiate the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [294]:
# Define a function to preprocess a text
def preprocess_text(text):
    # Tokenization: Split the text into words
    words = word_tokenize(text)

    # Remove punctuation and convert to lowercase
    words = [word.strip(string.punctuation).lower() for word in words]

    # Remove stop words
    words = [word for word in words if word not in stop_words]

    # Perform stemming
    #words = [stemmer.stem(word) for word in words]

    # Perform lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    # Remove empty strings
    words = [word for word in words if word]

    # Check if the first word is a numeric character and remove it, because there are leading numbers like 1,2,3,...
    if words and words[0].isdigit():
        words = words[1:]

    return words

In [295]:
# Preprocess the positive and negative reviews
preprocessed_positive_reviews = [preprocess_text(review) for review in positive_reviews]
preprocessed_negative_reviews = [preprocess_text(review) for review in negative_reviews]

In [296]:
# Print the preprocessed reviews
print("Preprocessed Positive Reviews:")
for review in preprocessed_positive_reviews:
    print(review)

Preprocessed Positive Reviews:
['forrest', 'gump', 'absolute', 'masterpiece', 'tom', 'hank', 'delivers', 'unforgettable', 'performance', 'storytelling', 'heartwarming', 'movie', 'journey', 'life', 'make', 'laugh', 'cry', 'appreciate', 'simple', 'beauty', 'existence']
['shawshank', 'redemption', 'timeless', 'classic', 'powerful', 'theme', 'hope', 'friendship', 'redemption', 'make', 'must-watch', 'morgan', 'freeman', 'tim', 'robbins', 'give', 'exceptional', 'performance', 'brilliantly', 'crafted', 'film']
['epic', 'conclusion', 'lord', 'ring', 'trilogy', 'return', 'king', 'cinematic', 'triumph', 'breathtaking', 'visuals', 'epic', 'battle', 'emotionally', 'resonant', 'story', 'make', 'monumental', 'achievement', 'filmmaking']
['la', 'la', 'land', 'love', 'letter', 'magic', 'hollywood', 'dream', 'chemistry', 'ryan', 'gosling', 'emma', 'stone', 'enchanting', 'music', 'dance', 'sequence', 'pure', 'delight', 'modern', 'musical', 'masterpiece']
['wes', 'anderson', 'whimsical', 'style', 'shine'

In [297]:
# Sample test movie review
test_review = "It's clear that the movie has both its enthusiasts and critics. While it may not be to everyone's taste, it's worth watching with an open mind to form your own opinion."

In [298]:
# Preprocess the test review using the same preprocessing function used for training data
preprocessed_test_review = preprocess_text(test_review)
preprocessed_test_review

['clear',
 'movie',
 'enthusiast',
 'critic',
 'may',
 'everyone',
 'taste',
 'worth',
 'watching',
 'open',
 'mind',
 'form',
 'opinion']

In [299]:
# Calculate the vocabulary size for Laplace smoothing
vocabulary_size_positive=len(set(word for review in preprocessed_positive_reviews for word in review))
vocabulary_size_positive

229

In [300]:
# Calculate the vocabulary size for Laplace smoothing
vocabulary_size_negative=len(set(word for review in preprocessed_negative_reviews for word in review))
vocabulary_size_negative

196

## Q 3: Choosing the correct N-Gram model

Since this is a small dataset, which contains only 12 positive and negative reviews, we can't go for high n values. Because larger N-grams require more data to estimate probabilities accurately. And also this is a sentiment analysis task, its better to use high N value for word prediction like task, because for those it can able to capture context and long dependencies. But for sentiment analysis small value of N is enough. And also due to small dataset we can't go for N>=3, it's better to go with unigram or bigram. (Additionally for this small dataset, if we go for trigram, probability of all words become 0 for this particular test review )


Eventhough I have done the calculation for both Uni-Gram and Bi-Gram -> Most suitable model is Bi-Gram model (N=2)

# Bigram

## Q 3: Implementation of Bi-Gram model

In [301]:
# Create a bigram model
def build_bigram_model(reviews):
    bigram_counts = defaultdict(lambda: defaultdict(int))
    unigram_counts = defaultdict(int)

    for review in reviews:
       # Add <S> at the beginning and </S> at the end of each review
        review = ['<S>'] + review + ['</S>']

        for i in range(len(review) - 1):
            bigram = (review[i], review[i + 1])
            bigram_counts[bigram[0]][bigram[1]] += 1
            unigram_counts[bigram[0]] += 1

    # Calculate bigram probabilities
    bigram_probabilities = defaultdict(lambda: defaultdict(float))
    bigram_count = defaultdict(lambda: defaultdict(int))
    for word1, next_words in bigram_counts.items():
        for word2, count in next_words.items():
            bigram_count[word1][word2] = count
            bigram_probabilities[word1][word2] = count / unigram_counts[word1]

    return bigram_probabilities, bigram_count

In [302]:
# Create bigram models for the positive and negative train sets
positive_train_bigram_model, positive_bigram_count = build_bigram_model(preprocessed_positive_reviews)
negative_train_bigram_model, negative_bigram_count = build_bigram_model(preprocessed_negative_reviews)

In [303]:
import pandas as pd

# Flatten the list of lists to get unique words
unique_words_positive = list(set(word for review in preprocessed_positive_reviews for word in review))

# Create a DataFrame for the bigram count matrix for the positive review
bigram_count_data = {word1: [positive_bigram_count[word1][word2] for word2 in unique_words_positive] for word1 in unique_words_positive}
bigram_count_df = pd.DataFrame(bigram_count_data, index=unique_words_positive)

# Print the bigram count matrix for the positive review
print("Bigram Count Matrix (Positive Review):")
bigram_count_df



Bigram Count Matrix (Positive Review):


,ability,hope,morgan,network,music,brilliance,resonant,drama,impact,charming,...,visuals,keep,pursuit,laugh,timeless,robbins,overcome,dream,kid,whimsical
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hope,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
morgan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
network,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
music,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
robbins,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
overcome,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dream,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
kid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Q 4: Bi-Gram probability

In [304]:
print("Bigram Probabilities for Positive Train Set:")
# Print the bigram probabilities for positive reviews
for word1, next_words in positive_train_bigram_model.items():
    for word2, probability in next_words.items():
        print(f'P({word2} | {word1}) : {probability}')


Bigram Probabilities for Positive Train Set:
P(forrest | <S>) : 0.08333333333333333
P(shawshank | <S>) : 0.08333333333333333
P(epic | <S>) : 0.08333333333333333
P(la | <S>) : 0.08333333333333333
P(wes | <S>) : 0.08333333333333333
P(inception | <S>) : 0.08333333333333333
P(social | <S>) : 0.08333333333333333
P(smith | <S>) : 0.08333333333333333
P(eternal | <S>) : 0.08333333333333333
P(princess | <S>) : 0.08333333333333333
P(fifty | <S>) : 0.08333333333333333
P(film | <S>) : 0.08333333333333333
P(gump | forrest) : 1.0
P(absolute | gump) : 1.0
P(masterpiece | absolute) : 1.0
P(tom | masterpiece) : 0.3333333333333333
P(</S> | masterpiece) : 0.3333333333333333
P(sci-fi | masterpiece) : 0.3333333333333333
P(hank | tom) : 1.0
P(delivers | hank) : 1.0
P(unforgettable | delivers) : 1.0
P(performance | unforgettable) : 1.0
P(storytelling | performance) : 0.5
P(brilliantly | performance) : 0.5
P(heartwarming | storytelling) : 1.0
P(movie | heartwarming) : 1.0
P(journey | movie) : 1.0
P(life | jou

In [305]:
print("Bigram Probabilities for Negative Train Set:")
for word1, next_words in negative_train_bigram_model.items():
    for word2, probability in next_words.items():
        print(f'P({word2} | {word1}) : {probability}')

Bigram Probabilities for Negative Train Set:
P(last | <S>) : 0.08333333333333333
P(another | <S>) : 0.08333333333333333
P(emoji | <S>) : 0.08333333333333333
P(fifty | <S>) : 0.16666666666666666
P(jack | <S>) : 0.08333333333333333
P(superman | <S>) : 0.08333333333333333
P(cat | <S>) : 0.08333333333333333
P(room | <S>) : 0.08333333333333333
P(battlefield | <S>) : 0.08333333333333333
P(gigli | <S>) : 0.08333333333333333
P(film | <S>) : 0.08333333333333333
P(airbender | last) : 1.0
P(disaster | airbender) : 1.0
P(film | disaster) : 0.5
P(convoluted | disaster) : 0.5
P(adaptation | film) : 0.14285714285714285
P(</S> | film) : 0.14285714285714285
P(ever | film) : 0.14285714285714285
P(remained | film) : 0.14285714285714285
P(divided | film) : 0.14285714285714285
P(content | film) : 0.14285714285714285
P(portrayal | film) : 0.14285714285714285
P(butcher | adaptation) : 0.5
P(dr | adaptation) : 0.5
P(beloved | butcher) : 1.0
P(animated | beloved) : 1.0
P(series | animated) : 0.5
P(film | anima

## Q 5: Bi-gram probability for the following test movie review

In order to deal with out of vocabulary (OOV) words, I have applied Add-K smoothing. To find better value for k, we need validation dataset.

In [306]:
# Define a function to calculate the probability of a review using bigram probabilities with Laplace smoothing
def calculate_review_probability(review, bigram_count, vocabulary_size, k=0.1):
    probability = 1.0  # Initialize the probability
    #k=0.1 laplaccian smoothing - Add one smoothing

    review = ['<S>'] + review + ['</S>']

    # Calculate the probability by multiplying individual bigram probabilities with Laplace smoothing
    for i in range(len(review) - 1):  # Use the length of the review
        current_word = review[i]
        next_word = review[i + 1]
        bigram = (current_word, next_word)

        # Print to check values
        print(f'Count({next_word} | {current_word}) : {bigram_count[current_word][next_word]}')

        # Apply Laplace smoothing by adding k to both the numerator and denominator
        current_word_count = sum(bigram_count[current_word].values())  # Sum of counts for the current word
        print(f'Count of {current_word}: {current_word_count}')
        probability *= (bigram_count[current_word][next_word] + k) / (k * vocabulary_size + current_word_count)

        # Print to check values
        print(f'Laplace P({next_word} | {current_word}) : {(bigram_count[current_word][next_word] + k) / (k * vocabulary_size + current_word_count)}')

        print()
    return probability

In [307]:
# Calculate the probability for the test review with respect to positive and negative datasets
positive_review_probability_bigram = calculate_review_probability(preprocessed_test_review, positive_bigram_count, vocabulary_size_positive)

Count(clear | <S>) : 0
Count of <S>: 12
Laplace P(clear | <S>) : 0.0028653295128939823

Count(movie | clear) : 0
Count of clear: 1
Laplace P(movie | clear) : 0.0041841004184100415

Count(enthusiast | movie) : 0
Count of movie: 1
Laplace P(enthusiast | movie) : 0.0041841004184100415

Count(critic | enthusiast) : 1
Count of enthusiast: 1
Laplace P(critic | enthusiast) : 0.04602510460251046

Count(may | critic) : 0
Count of critic: 1
Laplace P(may | critic) : 0.0041841004184100415

Count(everyone | may) : 0
Count of may: 1
Laplace P(everyone | may) : 0.0041841004184100415

Count(taste | everyone) : 0
Count of everyone: 1
Laplace P(taste | everyone) : 0.0041841004184100415

Count(worth | taste) : 0
Count of taste: 0
Laplace P(worth | taste) : 0.004366812227074236

Count(watching | worth) : 0
Count of worth: 0
Laplace P(watching | worth) : 0.004366812227074236

Count(open | watching) : 0
Count of watching: 0
Laplace P(open | watching) : 0.004366812227074236

Count(mind | open) : 1
Count of 

In [308]:
negative_review_probability_bigram = calculate_review_probability(preprocessed_test_review, negative_bigram_count, vocabulary_size_negative)

Count(clear | <S>) : 0
Count of <S>: 12
Laplace P(clear | <S>) : 0.0031645569620253164

Count(movie | clear) : 0
Count of clear: 0
Laplace P(movie | clear) : 0.00510204081632653

Count(enthusiast | movie) : 0
Count of movie: 4
Laplace P(enthusiast | movie) : 0.00423728813559322

Count(critic | enthusiast) : 1
Count of enthusiast: 1
Laplace P(critic | enthusiast) : 0.05339805825242719

Count(may | critic) : 0
Count of critic: 1
Laplace P(may | critic) : 0.0048543689320388345

Count(everyone | may) : 0
Count of may: 3
Laplace P(everyone | may) : 0.004424778761061947

Count(taste | everyone) : 0
Count of everyone: 0
Laplace P(taste | everyone) : 0.00510204081632653

Count(worth | taste) : 0
Count of taste: 0
Laplace P(worth | taste) : 0.00510204081632653

Count(watching | worth) : 0
Count of worth: 0
Laplace P(watching | worth) : 0.00510204081632653

Count(open | watching) : 0
Count of watching: 0
Laplace P(open | watching) : 0.00510204081632653

Count(mind | open) : 1
Count of open: 1
La

## Q 6: Predict the category of the test movie review using Bi-Gram model

In [309]:
# Print the probabilities
print("Positive Review Probability:", positive_review_probability_bigram)
print("Negative Review Probability:", negative_review_probability_bigram)

Positive Review Probability: 4.7559300599267804e-32
Negative Review Probability: 3.413542509672997e-31


In [310]:
# Compare the probabilities and make a classification decision based on the task
if positive_review_probability_bigram > negative_review_probability_bigram:
    print("Predicted: Positive")
else:
    print("Predicted: Negative")

Predicted: Negative


That is the Negative review, since it has the high probability of being assigned in negative class according to bigram model.

## Q 7:  Perplexity

I have explained the concept of perplexity in the pdf document which I have submitted for Part A answer

In [311]:
# Convert positive reviews to a set of tuples (converted sentences)
positive_reviews_set = {tuple(review) for review in preprocessed_positive_reviews}

# Convert the preprocessed test review to a tuple
test_review_tuple = tuple(preprocessed_test_review)

# Check for the overlap between train data and test data - Since having same sentence will give low value for perplexity
if test_review_tuple in positive_reviews_set:
    print("Test review overlaps with positive reviews in the training dataset.")
else:
    print("No overlap found.")

No overlap found.


In [312]:
# Convert negative reviews to a set of tuples (converted sentences)
negative_reviews_set = {tuple(review) for review in preprocessed_negative_reviews}

# Convert the preprocessed test review to a tuple
test_review_tuple = tuple(preprocessed_test_review)

# Check for the overlap between train data and test data - Since having same sentence will give low value for perplexity
if test_review_tuple in negative_reviews_set:
    print("Test review overlaps with positive reviews in the training dataset.")
else:
    print("No overlap found.")

No overlap found.


In [313]:
def calculate_perplexity(review, bigram_count, vocabulary_size, k=0.1):
    probability = 1.0
    word_count = 0

    review = ['<S>'] + review + ['</S>']

    for i in range(len(review) - 1):
        current_word = review[i]
        next_word = review[i + 1]

        # Apply Laplace smoothing by adding k to both the numerator and denominator
        current_word_count = sum(bigram_count[current_word].values())  # Sum of counts for the current word

        # Compute the conditional probability using Laplace smoothing
        conditional_probability = (bigram_count[current_word][next_word] + k) / (k * vocabulary_size + current_word_count)

        # Update perplexity
        probability *= conditional_probability

        # Exclude <S> from the word count
        if current_word != '<S>':
            word_count += 1

    # Compute perplexity as the inverse probability normalized by the word count
    perplexity = math.pow(1 / probability, 1 / word_count)

    return perplexity


In [314]:
# Calculate perplexity for the positive and negative test reviews
perplexity_positive = calculate_perplexity(preprocessed_test_review, positive_bigram_count, vocabulary_size_positive)
perplexity_negative = calculate_perplexity(preprocessed_test_review, negative_bigram_count, vocabulary_size_negative)

print(f"Perplexity for positive test review: {perplexity_positive}")
print(f"Perplexity for negative test review: {perplexity_negative}")

Perplexity for positive test review: 256.7103780050521
Perplexity for negative test review: 220.5969944806656


Minimizing Perplexity is same as Maximizing probability of the sentence.

## Prdeicting test review using Backoff smoothing

In [315]:
def calculate_review_probability(review, bigram_count, vocabulary_size):
    probability = 1.0  # Initialize the probability

    review = ['<S>'] + review + ['</S>']

    # Calculate the probability by multiplying individual bigram probabilities
    for i in range(len(review) - 1):
        current_word = review[i]
        next_word = review[i + 1]
        bigram = (current_word, next_word)

        # Check if the bigram exists in the bigram count dictionary
        if bigram in bigram_count[current_word]:
            probability *= bigram_count[current_word][bigram] / sum(bigram_count[current_word].values())
        else:
            # Backoff to unigram probability
            probability *= 1 / vocabulary_size

    return probability


In [316]:
# Calculate the probability for the test review with respect to positive and negative datasets
positive_review_probability_bigram = calculate_review_probability(preprocessed_test_review, positive_bigram_count, vocabulary_size_positive)
negative_review_probability_bigram = calculate_review_probability(preprocessed_test_review, negative_bigram_count, vocabulary_size_negative)

In [317]:
# Print the probabilities
print("Positive Review Probability:", positive_review_probability_bigram)
print("Negative Review Probability:", negative_review_probability_bigram)

Positive Review Probability: 9.168604668478727e-34
Negative Review Probability: 8.098694206237001e-33


In [318]:
# Compare the probabilities and make a classification decision based on the task
if positive_review_probability_bigram > negative_review_probability_bigram:
    print("Predicted: Positive")
else:
    print("Predicted: Negative")

Predicted: Negative


In [334]:
def calculate_perplexity(review, bigram_count, vocabulary_size, k=0.1):
    probability = 1.0
    word_count = 0

    review = ['<S>'] + review + ['</S>']

    for i in range(len(review) - 1):
        current_word = review[i]
        next_word = review[i + 1]
        bigram = (current_word, next_word)

        # Check if the bigram exists in the bigram count dictionary
        if bigram in bigram_count[current_word]:
            probability *= bigram_count[current_word][bigram] / sum(bigram_count[current_word].values())
        else:
            # Backoff to unigram probability
            probability *= 1 / vocabulary_size


        # Exclude <S> from the word count
        if current_word != '<S>':
            word_count += 1

    # Compute perplexity as the inverse probability normalized by the word count
    perplexity = math.pow(1 / probability, 1 / word_count)

    return perplexity


In [335]:
# Calculate perplexity for the positive and negative test reviews
perplexity_positive = calculate_perplexity(preprocessed_test_review, positive_bigram_count, vocabulary_size_positive)
perplexity_negative = calculate_perplexity(preprocessed_test_review, negative_bigram_count, vocabulary_size_negative)

print(f"Perplexity for positive test review: {perplexity_positive}")
print(f"Perplexity for negative test review: {perplexity_negative}")

Perplexity for positive test review: 347.8253963413587
Perplexity for negative test review: 294.15989474319053


By using Backoff smoothing, it has the high perplexity value compare to the method using Add-K smoothing.

Having low perplexity is good to maximize the probability. So this smoothing technique is not good.

# Unigram

## Q 3: Implementation of Uni-Gram model

In [319]:
def build_unigram_model(reviews):
    unigram_counts = defaultdict(int)
    total_words = 0

    for review in reviews:

        for word in review:
            unigram_counts[word] += 1
            total_words += 1

    # Calculate unigram probabilities
    unigram_probabilities = defaultdict(float)
    unigram_count = defaultdict(int)
    for word, count in unigram_counts.items():
        unigram_count[word] = count
        unigram_probabilities[word] = count / total_words

    return unigram_probabilities, unigram_count

In [320]:
# Create unigram models for the positive and negative train sets
positive_train_unigram_model, positive_unigram_count = build_unigram_model(preprocessed_positive_reviews)
negative_train_unigram_model, negative_unigram_count = build_unigram_model(preprocessed_negative_reviews)

In [321]:
print("Unigram Counts:")
for word, count in positive_unigram_count.items():
    print(f"{word}: {count}")

Unigram Counts:
forrest: 1
gump: 1
absolute: 1
masterpiece: 3
tom: 1
hank: 1
delivers: 1
unforgettable: 1
performance: 2
storytelling: 1
heartwarming: 1
movie: 1
journey: 2
life: 1
make: 5
laugh: 1
cry: 1
appreciate: 1
simple: 1
beauty: 1
existence: 1
shawshank: 1
redemption: 2
timeless: 2
classic: 2
powerful: 1
theme: 1
hope: 1
friendship: 1
must-watch: 1
morgan: 1
freeman: 1
tim: 1
robbins: 1
give: 1
exceptional: 1
brilliantly: 1
crafted: 1
film: 8
epic: 2
conclusion: 1
lord: 1
ring: 1
trilogy: 1
return: 1
king: 1
cinematic: 2
triumph: 1
breathtaking: 1
visuals: 1
battle: 1
emotionally: 1
resonant: 1
story: 2
monumental: 1
achievement: 1
filmmaking: 1
la: 2
land: 1
love: 3
letter: 1
magic: 1
hollywood: 1
dream: 2
chemistry: 2
ryan: 1
gosling: 1
emma: 1
stone: 1
enchanting: 1
music: 1
dance: 1
sequence: 1
pure: 1
delight: 2
modern: 2
musical: 1
wes: 1
anderson: 1
whimsical: 1
style: 1
shine: 2
grand: 1
budapest: 1
hotel: 1
quirky: 1
character: 2
colorful: 1
cinematography: 1
visual: 2

## Q 4: Uni-Gram probability

In [322]:
# Print unigram probabilities for each word
for word, probability in positive_train_unigram_model.items():
    print(f"{word}: {probability}")

forrest: 0.0037174721189591076
gump: 0.0037174721189591076
absolute: 0.0037174721189591076
masterpiece: 0.011152416356877323
tom: 0.0037174721189591076
hank: 0.0037174721189591076
delivers: 0.0037174721189591076
unforgettable: 0.0037174721189591076
performance: 0.007434944237918215
storytelling: 0.0037174721189591076
heartwarming: 0.0037174721189591076
movie: 0.0037174721189591076
journey: 0.007434944237918215
life: 0.0037174721189591076
make: 0.01858736059479554
laugh: 0.0037174721189591076
cry: 0.0037174721189591076
appreciate: 0.0037174721189591076
simple: 0.0037174721189591076
beauty: 0.0037174721189591076
existence: 0.0037174721189591076
shawshank: 0.0037174721189591076
redemption: 0.007434944237918215
timeless: 0.007434944237918215
classic: 0.007434944237918215
powerful: 0.0037174721189591076
theme: 0.0037174721189591076
hope: 0.0037174721189591076
friendship: 0.0037174721189591076
must-watch: 0.0037174721189591076
morgan: 0.0037174721189591076
freeman: 0.0037174721189591076
tim:

In [323]:
# Print unigram probabilities for each word
for word, probability in negative_train_unigram_model.items():
    print(f"{word}: {probability}")

last: 0.004149377593360996
airbender: 0.004149377593360996
disaster: 0.008298755186721992
film: 0.029045643153526972
adaptation: 0.008298755186721992
butcher: 0.004149377593360996
beloved: 0.004149377593360996
animated: 0.008298755186721992
series: 0.004149377593360996
wooden: 0.004149377593360996
acting: 0.012448132780082987
convoluted: 0.008298755186721992
storytelling: 0.004149377593360996
cringe-worthy: 0.004149377593360996
special: 0.012448132780082987
effect: 0.012448132780082987
letdown: 0.004149377593360996
fan: 0.008298755186721992
newcomer: 0.004149377593360996
alike: 0.004149377593360996
another: 0.004149377593360996
transformer: 0.004149377593360996
movie: 0.016597510373443983
mindless: 0.004149377593360996
explosion: 0.004149377593360996
incoherent: 0.004149377593360996
plotlines: 0.004149377593360996
overreliance: 0.004149377593360996
cgi: 0.004149377593360996
franchise: 0.004149377593360996
desperately: 0.004149377593360996
need: 0.004149377593360996
overhaul: 0.00414937

## Q 5: Uni-gram probability for the following test movie review

In [324]:
def calculate_review_probability_unigram(review, unigram_count, vocabulary_size):
    probability = 1.0  # Initialize the probability
    k = 0.5  # Laplace smoothing - Add one smoothing

    total_words = sum(unigram_count.values())
    print(f'Total Words: {total_words}')
    print(f'Unique Words: {vocabulary_size}')

    print()

    # Calculate the probability by multiplying individual unigram probabilities with Laplace smoothing
    for word in review:  # Use the length of the review
        # Print to check values
        print(f'Initial P({word}): {unigram_count[word]}')

        # Apply Laplace smoothing by adding k to both the numerator and denominator
        probability *= (unigram_count[word] + k) / (k * vocabulary_size + total_words)

        # Print to check values
        print(f'Laplace P({word}): {(unigram_count[word] + k) / (k * vocabulary_size + total_words)}')

        print()
    return probability


In [325]:
# Calculate the probability for the test review with respect to positive and negative datasets
positive_review_probability_unigram = calculate_review_probability_unigram(preprocessed_test_review, positive_unigram_count, vocabulary_size_positive)

Total Words: 269
Unique Words: 229

Initial P(clear): 1
Laplace P(clear): 0.003911342894393742

Initial P(movie): 1
Laplace P(movie): 0.003911342894393742

Initial P(enthusiast): 1
Laplace P(enthusiast): 0.003911342894393742

Initial P(critic): 1
Laplace P(critic): 0.003911342894393742

Initial P(may): 1
Laplace P(may): 0.003911342894393742

Initial P(everyone): 1
Laplace P(everyone): 0.003911342894393742

Initial P(taste): 0
Laplace P(taste): 0.001303780964797914

Initial P(worth): 0
Laplace P(worth): 0.001303780964797914

Initial P(watching): 0
Laplace P(watching): 0.001303780964797914

Initial P(open): 1
Laplace P(open): 0.003911342894393742

Initial P(mind): 2
Laplace P(mind): 0.00651890482398957

Initial P(form): 0
Laplace P(form): 0.001303780964797914

Initial P(opinion): 1
Laplace P(opinion): 0.003911342894393742



In [326]:
# Calculate the probability for the test review with respect to positive and negative datasets
negative_review_probability_unigram = calculate_review_probability_unigram(preprocessed_test_review, negative_unigram_count, vocabulary_size_positive)

Total Words: 241
Unique Words: 229

Initial P(clear): 0
Laplace P(clear): 0.0014064697609001407

Initial P(movie): 4
Laplace P(movie): 0.012658227848101266

Initial P(enthusiast): 1
Laplace P(enthusiast): 0.004219409282700422

Initial P(critic): 1
Laplace P(critic): 0.004219409282700422

Initial P(may): 3
Laplace P(may): 0.009845288326300985

Initial P(everyone): 0
Laplace P(everyone): 0.0014064697609001407

Initial P(taste): 0
Laplace P(taste): 0.0014064697609001407

Initial P(worth): 0
Laplace P(worth): 0.0014064697609001407

Initial P(watching): 0
Laplace P(watching): 0.0014064697609001407

Initial P(open): 1
Laplace P(open): 0.004219409282700422

Initial P(mind): 1
Laplace P(mind): 0.004219409282700422

Initial P(form): 0
Laplace P(form): 0.0014064697609001407

Initial P(opinion): 1
Laplace P(opinion): 0.004219409282700422



## Q 6: Predict the category of the test movie review using Uni-Gram model

In [327]:
# Print the probabilities
print("Positive Review Probability:", positive_review_probability_unigram)
print("Negative Review Probability:", negative_review_probability_unigram)

Positive Review Probability: 1.0318113735731843e-33
Negative Review Probability: 1.290155367564122e-33


In [328]:
# Compare the probabilities and make a classification decision based on the task
if positive_review_probability_unigram > negative_review_probability_unigram:
    print("Predicted: Positive")
else:
    print("Predicted: Negative")

Predicted: Negative


For Unigram model, according to the K value in add-k smoothing, model prediction changes.

When K=0.5, model prediction is negative (that means model says most of words are coincide with Negative review)

Positive Review Probability: 1.1624207195736878e-33

Negative Review Probability: 4.075676951702222e-33



But When K=0.1, model prediction is positive (that means model says most of words are coincide with positive review)

Positive Review Probability: 4.1578896371040005e-36

Negative Review Probability: 2.5647538549341524e-36


## Q 7: Perplexity

I have explained the concept of perplexity in the pdf document which I have submitted for Part A answer

In [329]:
def calculate_unigram_perplexity(review, unigram_count, vocabulary_size, k=0.5):
    probability = 1.0
    word_count = sum(unigram_count.values())

    for i in range(1, len(review)):  # Start from 1 to skip <S>
        current_word = review[i]

        # Apply Laplace smoothing by adding k to both the numerator and denominator
        current_word_count = unigram_count[current_word]

        # Update perplexity
        probability *= (current_word_count + k) / (k * vocabulary_size + word_count)

    # Compute perplexity as the inverse probability normalized by the word count
    perplexity = math.pow(1 / probability, 1 / word_count)

    return perplexity


In [330]:
# Calculate perplexity for the positive and negative test reviews
perplexity_positive = calculate_unigram_perplexity(preprocessed_test_review, positive_unigram_count, vocabulary_size_positive)
perplexity_negative = calculate_unigram_perplexity(preprocessed_test_review, negative_unigram_count, vocabulary_size_negative)

print(f"Perplexity for positive test review: {perplexity_positive}")
print(f"Perplexity for negative test review: {perplexity_negative}")

Perplexity for positive test review: 1.299198647475049
Perplexity for negative test review: 1.3292576489765537


Minimizing Perplexity is same as Maximizing probability of the sentence.